In [1]:
from os import listdir
from os.path import isfile, join
import cv2
import numpy as np

import matplotlib.pyplot as plt

# 1. Construir Datasets de Treino e Validação

In [2]:
def get_dataset(opt):
    
    DIR ="/home/lucas/Desktop/poli/PSI3472/rede_colorir_rede/dados/"+opt+"/"
    
    X = []
    Y = []
    
    imagens_treino = [f for f in listdir(DIR) if isfile(join(DIR, f))]
        
    i=1
    
    for imagem in imagens_treino:
        
        if i % 2 == 1:
            
            # Armazenar imagem colorida (Y)
            
            img = cv2.imread(DIR+imagem)
            
            img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
            
            Y.append(img)
            
        if i % 2 == 0:
            
            # Armazenar imagem preto e branca (X)
            
            img = cv2.imread(DIR+imagem, 0)
                        
            X.append(img)
            
        i=i+1
          
    X = np.array(X)
    Y = np.array(Y)
        
    return X, Y

In [3]:
def pre_process(X, Y):
    
    n = X.shape[0]
    n_linhas = X.shape[1]
    n_colunas = X.shape[2]
    
    X = np.float64(X)/255.0
    Y = np.float64(Y)/255.0
    
    # preto e branco
    X = X.reshape(n, n_linhas, n_colunas, 1)
    
    #colorido
    Y = Y.reshape(n, n_linhas, n_colunas, 3)
    
    input_shape = (n_linhas, n_colunas, 1)
    
    return X, Y, input_shape

In [4]:
X_treino, Y_treino = get_dataset("treino")

X_val, Y_val = get_dataset("validação")

In [5]:
X_treino, Y_treino, input_shape = pre_process(X_treino, Y_treino)

X_val, Y_val, input_shape = pre_process(X_val, Y_val)

# 2. Arquitetura da Rede Neural Convolucional do tipo U

<br>

<img src=https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41598-020-62484-z/MediaObjects/41598_2020_62484_Fig1_HTML.png?as=webp>

In [6]:
from keras.models import *
from keras.layers import *

In [7]:
def unet(n, input_shape):  

    inputs = Input(input_shape)
    
    conv2 = Conv2D(n,3, activation ='relu', padding ='same')(inputs)
    conv2 = Conv2D(n,3, activation ='relu', padding ='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
    
    conv3 = Conv2D(2*n,3, activation ='relu', padding ='same')(pool2)
    conv3 = Conv2D(2*n,3, activation ='relu', padding ='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2,2))(conv3)
                                                               
    conv4 = Conv2D(4*n,3, activation ='relu', padding ='same')(pool3)#8x8
    conv4 = Conv2D(4*n,3, activation ='relu', padding ='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2,2))(conv4)
    
    conv5 = Conv2D(8*n,3, activation ='relu', padding ='same')(pool4)
    conv5 = Conv2D(8*n,3, activation ='relu', padding ='same')(conv5)
    
    up6 = Conv2D(4*n,2, activation ='relu', padding ='same')(UpSampling2D(size =(2,2))(conv5))
    
    merge6 = concatenate([conv4,up6], axis =3)
    conv6 = Conv2D(4*n,3, activation ='relu', padding ='same')(merge6)
    conv6 = Conv2D(4*n,3, activation ='relu', padding ='same')(conv6)
    
    up7 = Conv2D(2*n,2, activation ='relu', padding ='same')(UpSampling2D(size =(2,2))(conv6))
    
    merge7 = concatenate([conv3,up7], axis =3)
    conv7 = Conv2D(2*n,3, activation ='relu', padding ='same')(merge7)
    conv7 = Conv2D(2*n,3, activation ='relu', padding ='same')(conv7)
    
    up8 = Conv2D(n,2, activation ='relu', padding ='same')(UpSampling2D(size =(2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis =3)
    conv8 = Conv2D(n,3, activation ='relu', padding ='same')(merge8)
    conv8 = Conv2D(n,3, activation ='relu', padding ='same')(conv8)
    
    conv9 = Conv2D(3,1, activation ='sigmoid', padding ='same')(conv8)
    
    model = Model(inputs = inputs, outputs = conv9)
                                                                           
    model.compile(optimizer = 'adam', loss ='mean_squared_error', metrics =['mse'])

    model.summary()

    return model

In [8]:
def fcnet(input_shape):
    
    model = Sequential()
    model.add(Conv2D(40, kernel_size=(5,5), strides=2, activation='relu', padding='same', input_shape=input_shape)) #saida “20”*16*16*40
    model.add(Dropout(0.25))
    model.add(Conv2D(12, kernel_size=(5,5), strides=2, activation='relu', padding='same')) #saida 8*8
    model.add(Dropout(0.25))
    model.add(Conv2D(12, kernel_size=(5,5), strides=2, activation='relu', padding='same')) #saida 4*4
    model.add(Dropout(0.25))
    model.add(Conv2DTranspose(12, kernel_size=(5,5), strides=2, activation='relu', padding='same')) #saida 8*8
    model.add(Dropout(0.25))
    model.add(Conv2DTranspose(40, kernel_size=(5,5), strides=2, activation='relu', padding='same')) #saida 16*16*40
    model.add(Dropout(0.25))
    model.add(Conv2DTranspose(3, kernel_size=(5,5), strides=2, padding='same')) #saida ?*32*32*1
                                                                               
    model.compile(optimizer = 'adam', loss ='mean_squared_error', metrics =['mse'])

    model.summary()

    return model    

In [9]:
#model = unet(30, input_shape)

In [ ]:
model = fcnet(input_shape)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 192, 128, 40)      1040      
_________________________________________________________________
dropout (Dropout)            (None, 192, 128, 40)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 64, 12)        12012     
_________________________________________________________________
dropout_1 (Dropout)          (None, 96, 64, 12)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 32, 12)        3612      
_________________________________________________________________
dropout_2 (Dropout)          (None, 48, 32, 12)        0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 96, 64, 12)        3

# 3. Treinar Rede Neural

In [ ]:
model.fit(X_treino, Y_treino, batch_size=10, epochs=1000, validation_data=(X_val, Y_val))

Epoch 1/1000
20/20 [==============================] - 16s 823ms/step - loss: 0.1280 - mse: 0.1280 - val_loss: 0.0531 - val_mse: 0.0531
Epoch 2/1000
20/20 [==============================] - 15s 740ms/step - loss: 0.0471 - mse: 0.0471 - val_loss: 0.0330 - val_mse: 0.0330
Epoch 3/1000
20/20 [==============================] - 15s 748ms/step - loss: 0.0286 - mse: 0.0286 - val_loss: 0.0244 - val_mse: 0.0244
Epoch 4/1000
20/20 [==============================] - 15s 764ms/step - loss: 0.0246 - mse: 0.0246 - val_loss: 0.0222 - val_mse: 0.0222
Epoch 5/1000
13/20 [==================>...........] - ETA: 4s - loss: 0.0228 - mse: 0.0228

In [ ]:
from keras.utils import plot_model
plot_model(model, show_shapes=True)

# 4. Avaliar o modelo

In [ ]:
score = model.evaluate(X_treino, Y_treino, verbose=0)
print('Training loss:', score)

score = model.evaluate(X_val, Y_val, verbose=0)
print('Validation loss:', score)

# 5. Salvando o Modelo

In [ ]:
model.save('psi3472.h5')